In [1]:
from IPython.display import display, Math

def print_matrix(array):
    matrix = ''
    for row in array:
        try:
            for number in row:
                matrix += f'{number}&'
        except TypeError:
            matrix += f'{row}&'
        matrix = matrix[:-1] + r'\\'
    display(Math(r'\begin{bmatrix}'+matrix+r'\end{bmatrix}'))

# Ingo's code

In [2]:
import numpy as np

MAGIC_BASIS = np.array(
    [[1, 0, 0, 1j], [0, 1j, 1, 0], [0, 1j, -1, 0], [1, 0, 0, -1j]]
) / np.sqrt(2)

HADAMARD = np.array([[1, 1, -1, 1], [1, 1, 1, -1], [1, -1, -1, -1], [1, -1, 1, 1]]) / 2

In [3]:
def orth_decomp_of_unitary(X):
    """Decomposes unitary X = Ql @ exp(i Theta) @ Qr'
    into orthogonal matrices Ql, Qr and angles Theta.

    """

    # one can also directly decompose XX^T and infer Ql from Qr. This might save about 3 eigh / qrs
    # I will here try to implement something where I understand the proof, why it works.
    # The benefit here is that we reliably work with real orthogonal matrices all the time.
    Xr = X.real
    Xi = X.imag
    _, Ql = np.linalg.eigh(Xr @ Xi.transpose())
    _, Qr = np.linalg.eigh(Xr.transpose() @ Xi)

    Dr = Ql.transpose() @ Xr @ Qr
    # if Xi injective this is diagonal, if not there is an arbitrary SO(dim ker(xi)) too much
    # fixing the kernels, I don't know if there is a smarted way to do this!
    if not np.allclose(Dr, np.diag(np.diag(Dr))):
        Q, R = np.linalg.qr(Dr)
        Dr = np.diag(R).copy()
        Ql = Ql @ Q
    else:
        Dr = np.diag(Dr).copy()

    Di = Ql.transpose() @ Xi @ Qr
    # if Xr injective this is diagonal, if not there is an arbitrary SO(dim ker(Xr)) too much
    if not np.allclose(Di, np.diag(np.diag(Di))):
        Q, R = np.linalg.qr(Di.T)
        Di = np.diag(R).copy()
        Qr = Qr @ Q

    else:
        Di = np.diag(Di).copy()

    # ensure Ql, Qr are in SO
    if np.linalg.det(Ql) < 0:
        Ql[:, 0] = -Ql[:, 0]
        Dr[0] = -Dr[0]
        Di[0] = -Di[0]

    if np.linalg.det(Qr) < 0:
        Qr[:, 0] = -Qr[:, 0]
        Dr[0] = -Dr[0]
        Di[0] = -Di[0]

    Theta = np.angle(Dr + 1j * Di)

    return Theta, Qr, Ql

def unit_kronecker_rank_approx(X, verbose=True):
    """Approximates a n^2 x m^2 matrix X  as A otimes B"""
    # better check for square dimensions

    n, m = np.sqrt(X.shape).astype(int)

    Y = X.reshape(n, n, m, m).transpose((0, 2, 1, 3)).reshape(n * m, n * m)
    U, S, Vh = np.linalg.svd(Y)

    # if verbose:
    #     print("Truncating the spectrum", S)

    U = U @ np.sqrt(np.diag(S))
    Vh = np.sqrt(np.diag(S)) @ Vh
    A = U[:, 0].reshape(n, m)
    B = Vh[0, :].reshape(n, m)
    return A, B

In [5]:
def KAK_decomp(U):
    """Calculates the KAK decomposition of U in U(4).
    U = np.kron(A0, A1) @ heisenberg_unitary(K) @ np.kron(B0.conj().T, B1.conj().T)
    """

    Theta, Qr, Ql = orth_decomp_of_unitary(MAGIC_BASIS.conj().T @ U @ MAGIC_BASIS)
    A0, A1 = unit_kronecker_rank_approx(MAGIC_BASIS @ Ql @ MAGIC_BASIS.conj().T)
    B0, B1 = unit_kronecker_rank_approx(MAGIC_BASIS @ Qr @ MAGIC_BASIS.conj().T)
    K = HADAMARD.T @ Theta / 2

    # print("Theta", Theta)
    # print("K", K)

    # Make A0, A1, B0, B1 special
    def make_special(X):
        return np.sqrt(np.linalg.det(X).conj()) * X

    A0, A1, B0, B1 = map(make_special, (A0, A1, B0, B1))

    return A0, A1, K, B0, B1

# Tests

In [6]:
PAULI_X = np.array([[0, 1], [1, 0]])
PAULI_Y = np.array([[0, -1j], [1j, 0]])
PAULI_Z = np.array([[1, 0], [0, -1]])

PXX = np.kron(PAULI_X, PAULI_X)
PYY = np.kron(PAULI_Y, PAULI_Y)
PZZ = np.kron(PAULI_Z, PAULI_Z)

In [10]:
# some convenient functions and constants
def haar_rand_unitary(dim, real=False):
    """Returns a random unitary dim x dim matrix drawn from the Haar measure.

    Args:
        dim (int): the dimension of the unitary
        real (boolean): If true returns an orthogonal matrix.

    Returns:
        numpy.array: random unitary matrix
    """

    # Draw a random Gaussian
    gaussianMatrix = np.random.randn(dim, dim)
    if not real:  # Add a random imaginary part
        gaussianMatrix = gaussianMatrix + 1j * np.random.randn(dim, dim)

    # project to Haar random unitary
    Q, R = np.linalg.qr(gaussianMatrix)
    D = np.diag(R)
    D = D / np.abs(D)
    R = np.diag(D)
    Q = Q.dot(R)

    return Q

def is_unitary(X, special=False, eps=1e-8):
    is_special = np.abs(np.linalg.det(X) - 1) < 1e-8 if special else True
    return is_special and np.linalg.norm(X @ X.conj().T - np.eye(X.shape[0])) < eps

def heisenberg_unitary(k):
    from scipy.linalg import expm

    H = k[0] * np.eye(4) + k[1] * PXX + k[2] * PYY + k[3] * PZZ
    return expm(1j * H)

## test low_kronecker_rank_approx

In [8]:
n, m = 2, 3

A = np.random.randn(n, m)
B = np.random.randn(n, m)

C = np.kron(A, B)

Arec, Brec = unit_kronecker_rank_approx(C)

assert np.linalg.norm(C - np.kron(Arec, Brec)) < 1e-8


## test orth_decomp_of_unitary

In [23]:
dim = 10

X = haar_rand_unitary(dim)


def test_orth_decomp_of_unitary(X):

    Theta, Qr, Ql = orth_decomp_of_unitary(X)
    assert np.linalg.norm(Ql @ np.diag(np.exp(1j * Theta)) @ Qr.T - X) < 1e-8


test_orth_decomp_of_unitary(X)


## test KAK_decomp(U)


In [24]:
def test_KAK_decomposition(U):
    A0, A1, K, B0, B1 = KAK_decomp(U)

    # Correctly decomposed
    assert (
        np.linalg.norm(
            np.kron(A0, A1) @ heisenberg_unitary(K) @ np.kron(B0.conj().T, B1.conj().T)
            - U
        )
        < 1e-8
    )

    # A0, A1, B0, B1 are in SU(2)
    assert is_unitary(A0, special=True)
    assert is_unitary(A1, special=True)
    assert is_unitary(B0, special=True)
    assert is_unitary(B1, special=True)

## test KAK_decomp of random unitary


In [25]:
test_KAK_decomposition(haar_rand_unitary(4))


Truncating the spectrum [2.00000000e+00 4.44592333e-16 2.32626279e-16 4.32850843e-17]
Truncating the spectrum [2.00000000e+00 3.89653131e-16 9.65150913e-17 6.90526114e-17]
Theta [2.07729384 3.01223401 1.32817007 0.74953761]
K [ 1.79180888  0.75295504  0.08907693 -0.37839316]


## test I, CNOT

In [28]:
tc = np.eye(4)

# Generates error
test_KAK_decomposition(tc)

Truncating the spectrum [2.00000000e+00 2.33376666e-16 6.50353591e-17 2.38194531e-33]
Truncating the spectrum [2.00000000e+00 2.33376666e-16 6.50353591e-17 2.38194531e-33]
Theta [ 3.14159265e+00 -5.15630170e-34  5.15630170e-34 -3.14159265e+00]
K [ 0.          1.57079633 -1.57079633  0.        ]


AssertionError: 

In [25]:
# function to print the reassembled matrix and the original matrix
def test_KAK_decomposition_2(U):
    A0, A1, K, B0, B1 = KAK_decomp(U)

    print("\nThe matrix reassembled after the KAK decomposition")
    after_kak = np.kron(A0, A1) @ heisenberg_unitary(K) @ np.kron(B0.conj().T, B1.conj().T)
    print_matrix(after_kak.round(5))

    print("The original matrix")
    print_matrix(U)

    dist = np.linalg.norm(after_kak - U)
    print("The distance between the two matrices is", dist)

In [26]:
# IDENTITY GATE
id = np.eye(4, dtype=int)
test_KAK_decomposition_2(id)


The matrix reassembled after the KAK decomposition


<IPython.core.display.Math object>

The original matrix


<IPython.core.display.Math object>

The distance between the two matrices is 4.000000000000003


In [29]:
# CNOT GATE
cnot = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
test_KAK_decomposition_2(cnot)


The matrix reassembled after the KAK decomposition


<IPython.core.display.Math object>

The original matrix


<IPython.core.display.Math object>

The distance between the two matrices is 2.8284271247461885


In [28]:
# CZ GATE
cz = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, -1]])
test_KAK_decomposition_2(cz)


The matrix reassembled after the KAK decomposition


<IPython.core.display.Math object>

The original matrix


<IPython.core.display.Math object>

The distance between the two matrices is 2.000000000000001
